In [1]:
from DataGenerator import TimeSeriesGenerator, suddenDrift, incrementalDrift
from conditions import scenarios
from sklearn.preprocessing import StandardScaler
from grouped_series import SeriesGrouper, ExperimentTracker, Experiment
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split

In [2]:
onehot_cols = ColumnTransformer([('one_hot_encoder', OneHotEncoder(
), make_column_selector(dtype_include=object))], remainder=StandardScaler())

xgb_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', XGBRegressor())
])

sgd_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', SGDRegressor())
])

In [3]:
def full_run():
    products = []
    customers = []
    for i in range(2):
        product = TimeSeriesGenerator(size=365, amountOfVariables=7)
        customer = TimeSeriesGenerator(size=365, amountOfVariables=3)
        products.append(product)
        customers.append(customer)
    thesis = ExperimentTracker(products, customers, scenarios())
    thesis.runExperiment(algorithm=sgd_pipe)
    return thesis


In [4]:
thesis = full_run()

Finished experiment! Elapsed time: 0.1536250114440918, total Elapsed time: 0.1536250114440918, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Important
Finished experiment! Elapsed time: 0.07581567764282227, total Elapsed time: 0.22944068908691406, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observed, importance: Important
Finished experiment! Elapsed time: 0.059853315353393555, total Elapsed time: 0.2892940044403076, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Unobserved, importance: Important
Finished experiment! Elapsed time: 0.05586719512939453, total Elapsed time: 0.34516119956970215, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Medium
Finished experiment! Elapsed time: 0.05386614799499512, total Elapsed time: 0.39902734756469727, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time

In [5]:
results = thesis.resultsToDF()

In [6]:
results[["SMAPE", "Drift type", "Drift time", "Drift magnitude"]].groupby(["Drift type", "Drift time", "Drift magnitude"]).aggregate(["mean", "std", "min", "max"])

SMAPE                  \
                                                      mean       std   min   
Drift type        Drift time     Drift magnitude                             
Incremental Drift Fully observed Large            3.033333  2.453555  1.82   
                                 Small            2.386667  0.373347  2.17   
                  Half observed  Large            7.271667  5.171669  2.33   
                                 Small            2.840833  0.948391  2.26   
                  Unobserved     Large            6.205833  4.168345  2.14   
                                 Small            2.745833  0.729426  2.26   
No Drift          No Drift       No Drift         2.315000  0.010000  2.31   
Sudden Drift      Fully observed Large            3.174167  2.792810  1.84   
                                 Small            2.430000  0.462247  2.19   
                  Half observed  Large            3.770000  4.372278  1.85   
                                 Small            2.525000  0.738814  2.19   
                  Unobserved     Large            7.661667  5.900542  2.34   
                                 Small            2.953333  1.080390  2.24   
Sudden Shock      3 days         Large            2.465000  0.256710  2.33   
                                 Small            2.342500  0.020616  2.32   
                  7 days         Large            2.452500  0.285117  2.30   
                                 Small            2.350000  0.048305  2.31   

                                                         
                                                    max  
Drift type        Drift time     Drift magnitude         
Incremental Drift Fully observed Large            10.31  
                                 Small             3.54  
                  Half observed  Large            20.01  
                                 Small             5.66  
                  Unobserved     Large            16.24  
                                 Small             4.96  
No Drift          No Drift       No Drift          2.33  
Sudden Drift      Fully observed Large            11.48  
                                 Small             3.87  
                  Half observed  Large            16.84  
                                 Small             4.83  
                  Unobserved     Large            22.85  
                                 Small             6.19  
Sudden Shock      3 days         Large             2.85  
                                 Small             2.37  
                  7 days         Large             2.88  
                                 Small             2.42

In [7]:
results.sort_values("SMAPE", ascending=False)   

,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,SMAPE
52,0.0,Sudden Drift,Large,Important,Unobserved,858.902932,22.85
69,0.0,Incremental Drift,Large,Important,Half observed,859.589639,20.01
51,0.0,Sudden Drift,Large,Important,Half observed,492.628037,16.84
70,0.0,Incremental Drift,Large,Important,Unobserved,720.642213,16.24
99,3.0,Sudden Drift,Large,Unimportant,Unobserved,179.208462,12.21
...,...,...,...,...,...,...,...
133,6.0,Sudden Drift,Large,Important,Half observed,9.387895,1.85
27,NaN,Incremental Drift,Large,Important,Fully observed,9.223463,1.84
132,6.0,Sudden Drift,Large,Important,Fully observed,9.327875,1.84
109,3.0,Incremental Drift,Large,Important,Fully observed,9.250843,1.83
